<a href="https://colab.research.google.com/github/ssonone/documents/blob/main/%5B%EC%8B%A4%EC%8A%B5%5D_11_(Colab)_%EC%98%A4%ED%94%88_%EB%AA%A8%EB%8D%B8%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_PEFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [실습] 오픈 모델을 이용한 PEFT

허깅페이스의 모델을 다운로드하고,   
데이터를 활용하여 LoRA 파인 튜닝을 수행합니다.   
파인 튜닝의 목적은 **'건방진 QA 봇 만들기'** 입니다.

### 중요) 코랩에서 실행하는 경우, T4 GPU 할당이 필요합니다.

## 1. 실습 환경 세팅하기

In [ ]:
!nvidia-smi

In [ ]:
!pip install seaborn langchain langchain-huggingface transformers bitsandbytes pandas peft accelerate datasets huggingface_hub trl --upgrade

In [ ]:
!pip install flash-attn --no-build-isolation
# flash attention
# Free Colab GPU: T4에서 적용 불가
# Windows 설치 불가


# READ 토큰 불러오기

https://huggingface.co/settings/tokens    
위 링크에서 READ 권한 토큰을 생성합니다.

In [ ]:
from huggingface_hub import login

# login(token='')
# Read 권한 토큰을 입력하세요!

## 2.모델 찾아서 저장하고 불러오기

In [ ]:
import torch
from transformers import AutoModelForCausalLM,AutoTokenizer,BitsAndBytesConfig
import transformers

model_id='qwen/qwen2.5-3b-instruct' # 모델의 주소
# huggingface.co/qwen/qwen2.5-3b-instruct



# 양자화 Configuration 설정 - BitsAndBytes
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


모델과 토크나이저를 불러옵니다.    

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             torch_dtype='auto',
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             # attn_implementation="flash_attention_2"
                                             # Free Colab GPU: T4에서 적용 불가
                                             )



In [ ]:
from transformers import pipeline

gen_config = dict(
    do_sample=True,
    max_new_tokens=512,
    repetition_penalty = 1.1,
    temperature = 0.7,
    top_p = 0.8,
    top_k = 20
)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, return_full_text=True,
                **gen_config)

## 파인 튜닝 데이터셋 준비하기

SFT 실습에 사용할 데이터를 준비합니다.

In [ ]:
file_path = 'RAG_Data_full.csv'
import pandas as pd
pd.read_csv(file_path).head()

datasets 라이브러리를 통해 데이터를 불러옵니다.

In [ ]:
from datasets import load_dataset
import os

file_path = ['RAG_Data_full.csv', 'RAG_Data_full_neg.csv']

data = load_dataset("csv",
                    data_files={"train":file_path})

# train_test split 나누기
# data = load_dataset("csv",
#                     data_files={"train":file_path}, split='train').train_test_split(0.1)

data = data.shuffle()
data

데이터를 포맷팅하여 LLM에 입력할 프롬프트로 변환합니다.

In [ ]:
def convert_format(context,question,answer=None, add_generation_prompt=False):
    # add_generation_prompt의 필요 여부에 따라 결정

    chat = [
        {'role':'system',
         'content':"""당신은 매우 거만합니다. [Context]를 참고하여, 사용자의 [Question]에 반말로 대답하세요.
정답을 알고 있는 경우, 답변은 항상 '그것도 몰라?' 로 시작해야 합니다. 사용자의 질문에 대한 답변을 하기 위해 필요한 본문의 일부를 인용하세요.
답을 모르는 경우, '내가 그딴 걸 어떻게 알아?' 라고만 답변하세요."""},
        {'role':'user',
         'content':f"Context: {context}\n"
         "---\n"
        f"Question: {question}"}
    ]
    if answer:
        chat.append(
            {'role':'assistant',
             'content':f"{answer} END"}
        )
    return {'text':tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=add_generation_prompt)}


In [ ]:
data = data.map(lambda x:convert_format(x['context'],x['question'], x['answer']))


In [ ]:
[row for row in data['train']][0]['text']

In [ ]:
print(data['train'][0]['text'])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import Counter

def analyze_token_distribution(dataset, text_column='text', bins=30):

    # 토큰 수 계산
    token_counts = []
    for text in dataset[text_column]:
        tokens = tokenizer.encode(text)
        token_counts.append(len(tokens))

    # 기본 통계 계산
    stats = {
        '평균 토큰 수': np.mean(token_counts),
        '중앙값': np.median(token_counts),
        '최소 토큰 수': min(token_counts),
        '최대 토큰 수': max(token_counts),
        '표준편차': np.std(token_counts),
        '90퍼센타일': np.percentile(token_counts, 90),
        '95퍼센타일': np.percentile(token_counts, 95),
        '99퍼센타일': np.percentile(token_counts, 99),
        '총 샘플 수': len(token_counts)
    }

    # 분포 시각화
    plt.figure(figsize=(12, 6))

    # 히스토그램과 KDE
    sns.histplot(data=token_counts, bins=bins, kde=True)
    plt.title(f'Token Length Distribution for {text_column}')
    plt.xlabel('Token Count')
    plt.ylabel('Frequency')

    plt.tight_layout()
    plt.show()

    # 상세 통계 출력
    print("\n=== 토큰 수 통계 ===")
    for key, value in stats.items():
        print(f"{key}: {value:.1f}")

    return stats

print(analyze_token_distribution(data['train']))

In [ ]:
test_context = '''OpenAI는 2018년에 최초의 GPT 모델(GPT-1)을 도입하여 "생성 사전 훈련을 통한 언어 이해 개선"이라는 논문을 발표했다.
이는 트랜스포머 아키텍처를 기반으로 하며 대규모 책 모음에서 훈련되었다.
다음 해에는 일관된 텍스트를 생성할 수 있는 더 큰 모델인 GPT-2를 도입했다.
2020년에는 GPT-2보다 100배 많은 매개변수를 갖고 몇 가지 예제만으로 다양한 작업을 수행할 수 있는 모델인 GPT-3을 출시했다.
GPT-3는 GPT-3.5로 더욱 개선되어 챗봇 제품인 ChatGPT를 만드는 데 사용되었다.
소문에 따르면 GPT-4에는 1조 7600억 개의 매개변수가 있는데, 이는 실행 속도와 조지 호츠에 의해 처음 추정되었다.'''

test_question = '''GPT-4의 파라미터는 몇 개인가요?'''

test_prompt = convert_format(test_context,test_question, add_generation_prompt=True)['text']
print(test_prompt)

In [ ]:
response = pipe(test_prompt, truncation=True)

print(response[0]['generated_text'])

## 6. PEFT(Prompt-Efficient Fine Tuning)로 학습하기   
전체 파인 튜닝을 하지 않고도, PEFT를 사용하면 파라미터의 수를 줄인 효과적인 튜닝이 가능합니다.

In [ ]:
model

트랜스포머 기반의 모델이 아닌 경우, LoRa를 적용하는 레이어가 달라질 수 있습니다.   
model의 출력 결과에서 모델의 구성 요소를 찾아봅시다.

In [ ]:
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r = 4, # 4차원 (d-->r) : 보통은 8이나 16
    lora_alpha=8, # (보통은 r과 함께 (8,16) 또는 (16,32))
    target_modules=[
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
    "up_proj",
    "down_proj",
    "gate_proj"],
    # LoRA를 어떤 모듈에 부착할 것인가? (경험적)
    # LoRA : Everything Except gate
    # QLoRA(베이스모델을 양자화하는 경우): Everything

    # Continuous Pretraining : embed_tokens 과 lm_head까지 부착 (메모리 소모 증가)
    # Ref) https://unsloth.ai/blog/contpretraining

    lora_dropout=0,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)

In [ ]:
model

In [ ]:
# 학습되는 파라미터 수 출력
model.print_trainable_parameters()

Huggingface의 trl 라이브러리는 파인 튜닝을 쉽게 수행하게 해 주는 라이브러리입니다.     
이번에 수행할 파인 튜닝은 Supervised Fine Tuning이므로, `SFTTrainer`를 불러와서 수행합니다.

In [ ]:
data['train']

In [ ]:
len(data['train']['text'])

In [ ]:
from trl import SFTTrainer, SFTConfig
from trl import DataCollatorForCompletionOnlyLM
from accelerate import Accelerator

accelerator = Accelerator()

tokenizer.pad_token = tokenizer.eos_token

response_template = "<|im_start|>assistant"
# 답변부분만 학습시키기

collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)
# Data Collator : 데이터를 어떻게 전달할 것인가?
# Next Token Prediction의 방법은 동일하나, 어디서부터 학습시킬 것인지를 결정
# 질문 내용까지 학습하기 VS 답변만 학습하기
# DataCollator를 전달하지 않으면, 전체 텍스트를 그대로 학습


sft_config = SFTConfig(
    report_to='none',

    # evaluation_strategy="steps",
    # # logging_step당 validation loss 계산하는 옵션

    max_steps= 500,
    # batch의 입력을 max_steps만큼 수행
    # batch * max_steps / len(data['train']) = Epochs
    dataset_text_field="text",
    # dataset 'text' 필드를 사용

    per_device_train_batch_size=1,
    # GPU당 데이터 1개씩 입력

    gradient_accumulation_steps=4,
    # (Batch 대용) 그래디언트를 모아서 반영하는 스텝 수
    # 배치사이즈를 키우는 것에 비해 메모리 소모가 감소하나, 속도가 느려짐

    max_seq_length=768,
    lr_scheduler_type='cosine',
    # 학습률을 cosine 형태로 점진적 감소

    learning_rate=1e-4,
    bf16=True, # bfloat16 모델이므로 bf16 설정

    optim="paged_adamw_8bit",
    output_dir="outputs",
    logging_steps=25,
    # 손실함수 출력

    # save_steps=50
    # 체크포인트 저장
)
trainer = SFTTrainer(
    model=model,
    train_dataset=data['train'],
    # # eval 데이터셋을 사용하고 싶은 경우 아래 주석 해제
    # eval_dataset=data['test'],

    args=sft_config,
    data_collator=collator

)

with accelerator.main_process_first():
    trainer.train()

# Loss Function : Cross Entropy
# 입력 데이터(배치)에 대한 평균 예측 확률 : e^(-Loss)
# Ex) 0.2 --> 81% (e^-0.2)

## 7. 학습 결과 확인하기

출력의 포맷을 END 토큰으로 끝맺었으므로, pipeline의 stop_sequence를 통해 출력을 제어합니다.

In [ ]:
gen_config = dict(
    do_sample=True,
    max_new_tokens=512,
    repetition_penalty = 1.1,
    temperature = 0.7,
    top_p = 0.8,
    top_k = 20
)

model.eval()

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, return_full_text=True,
                **gen_config)

In [ ]:
test_context = '''참고래의 등 부분은 밤회색이며, 배 쪽은 하얗다.
튀어나온 두 쌍의 숨구멍이 있으며, 납작하고 넓은 주둥이를 가지고 있다.
두 개의 밝은 색 문양이 숨구멍 뒤에서 시작해 몸의 측면으로 따라가 꼬리로 이어진다.
오른쪽 턱에 하얀색 무늬가 있으며, 왼쪽은 회색 또는 검은색이다.
참고래는 턱에서 몸 밑의 중앙부까지 이어지는 56에서 100개의 주름을 지니고 있는데,
먹이를 잡을 때 목을 팽창시키기 쉽게 하기 위한 것이다.
이들의 등지느러미의 길이는 60센티미터 정도이다.
가슴지느러미는 아주 작으며, 꼬리는 넓고 V자 모양이며 끝은 뾰족한 편이다.'''

test_question = '''참고래의 주름은 어떤 용도인가요?'''

test_prompt = convert_format(test_context,test_question, add_generation_prompt=True)['text']

print(test_prompt)

In [ ]:
model.eval()

response = pipe(test_prompt, stop_sequence=" END")

print(response[0]['generated_text'])

In [ ]:
test_context = '''참고래의 등 부분은 밤회색이며, 배 쪽은 하얗다.
튀어나온 두 쌍의 숨구멍이 있으며, 납작하고 넓은 주둥이를 가지고 있다.
두 개의 밝은 색 문양이 숨구멍 뒤에서 시작해 몸의 측면으로 따라가 꼬리로 이어진다.
오른쪽 턱에 하얀색 무늬가 있으며, 왼쪽은 회색 또는 검은색이다.
참고래는 턱에서 몸 밑의 중앙부까지 이어지는 56에서 100개의 주름을 지니고 있는데,
먹이를 잡을 때 목을 팽창시키기 쉽게 하기 위한 것이다.
이들의 등지느러미의 길이는 60센티미터 정도이다.
가슴지느러미는 아주 작으며, 꼬리는 넓고 V자 모양이며 끝은 뾰족한 편이다.'''

test_question = '''참고래는 무슨 색인가요?'''

test_prompt = convert_format(test_context,test_question, add_generation_prompt=True)['text']

print(test_prompt)

In [ ]:
response = pipe(test_prompt, stop_sequence=" END")

print(response[0]['generated_text'])

## 8. 모델 저장하고 huggingface 계정에 업로드하기

WRITE 권한 토큰이 필요합니다.

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

login('')

# Write 권한 토큰 필요

In [ ]:
# model.save_pretrained('./Qwen2.5_Rude_RAG')

In [ ]:
# # # 개인 계정 주소에 업로드하기

# model.push_to_hub('NotoriousH2/Qwen2.5_Rude_RAG')
# tokenizer.push_to_hub('NotoriousH2/Qwen2.5_Rude_RAG')
# # # 토크나이저도 함께 업로드


만약 베이스모델이 포함된 전체 모델을 업로드하고 싶다면 `merge_and_unload()`를 사용합니다.

In [ ]:
model = model.merge_and_unload()
model

In [ ]:
# # # 개인 계정 주소에 업로드하기

# model.push_to_hub('NotoriousH2/Qwen2.5_Rude_RAG_FULL')
# tokenizer.push_to_hub('NotoriousH2/Qwen2.5_Rude_RAG_FULL')

In [ ]:
# # 이후 작업을 위해 오프라인 저장
# model.save_pretrained('./Qwen2_Rude_RAG_FULL')

모델을 저장한 뒤, Kernel Restart를 실행하여 공간을 비웁니다.